# Student Name: Ghiyas Nizamudden Shaik

# Instructor Name: Yasser Abduallah

## C634 Mid Term Project: Implementation of Supervised Data Mining

#### Note: This programs assumes the following software and packages are installed:

`python`: 3.11.7

`pandas`: 2.2.3

`mlxtend`: 0.23.4

### Import the required packages

In [ ]:
import pandas as pd
import time
from mlxtend.frequent_patterns import apriori, association_rules
pd.set_option('display.max_colwidth', None)

### List the shops used in the program

In [ ]:
shop_names = ["fashionhub","grocerymart","sportszone","homessentials","techworld"]

### Choosing the database and formatting data

In [ ]:
# prompt the user for database
flag = True
while flag:
    shop_no = input("1. Fashion Hub\n2. Grocery Mart\n3. Sports Zone\n4. Home Essentials\n5. Tech World\nPlease select shop number from the following options - ")
    try:
        shop_no = int(shop_no)
        if shop_no in [1,2,3,4,5]:
            flag = False
        else:
            print(f"{shop_no} is an invalid input! Please select the shop number 1,2,3,4,5.")
    except:
        print(f"{shop_no} is an invalid input! Please select the shop number 1,2,3,4,5.")


data = pd.read_csv(f"data/{shop_names[shop_no-1]}_trans.csv",sep="|",index_col=0)

# format the data and extract a list of all the items present in the shop
def format_data(shop_data):
    df = shop_data["Transaction"].str.split(",", expand=True).map(lambda x: x.strip() if isinstance(x, str) else x).values
    shop_trans = [list(filter(lambda x: x is not None, row)) for row in df]
    shop_items = list(frozenset([item for trans in shop_trans for item in trans]))
    return {"items":shop_items, "trans":shop_trans}

shop_data = format_data(data)

1. Fashion Hub
2. Grocery Mart
3. Sports Zone
4. Home Essentials
5. Tech World
Please select shop number from the following options -  2


### Choosing minimum support

In [ ]:
# prompt the user for minimum support

flag = True
while flag:
    min_support = input("Please provide the minimum support (0,100]- ")
    try:
        min_support = float(min_support)
        if min_support <= 100 and min_support > 0:
            flag = False
        else:
            print(f"{min_support} is an invalid input! Please provide a number in the range (0,100].")
    except:
        print(f"{min_support} is an invalid input! Please provide a number in the range (0,100].")

min_support /= 100

Please provide the minimum support (0,100]-  40


### Choosing minimum confidence

In [ ]:
# get minimum confidence

flag = True
while flag:
    min_conf = input("Please provide the minimum confidence (0,100]- ")
    try:
        min_conf = float(min_conf)
        if min_conf <= 100 and min_conf > 0:
            flag = False
        else:
            print(f"{min_conf} is an invalid input! Please provide a number in the range (0,100].")
    except:
        print(f"{min_conf} is an invalid input! Please provide a number in the range (0,100].")

min_conf /= 100

Please provide the minimum confidence (0,100]-  60


### Brute Force Implementation

In [ ]:
# returns all the possible combinations of k-length itemsets from the given a list of items

def get_combinations(items,k,start=0,subset=None,result=None):
    if subset is None:
        subset = []
    if result is None:
        result = []
    if len(subset) == k:
        result.append(frozenset(subset))
        return result

    for i in range(start, len(items)):
        get_combinations(items, k, i + 1, subset + [items[i]], result)
    return result

In [ ]:
# filters the frequent itemsets from the given list of itemsets based on minimum support provided

def get_k_freq_itemsets(shop_data,itemsets,min_support):
    itemsets_count = {itemset:0 for itemset in itemsets}
    for trans in shop_data["trans"]:
        for itemset in itemsets:
            if itemset.issubset(trans):
                itemsets_count[itemset] += 1

    result = {itemset:count for itemset,count in itemsets_count.items() if count >= min_support*len(shop_data["trans"])}
    return result

In [ ]:
# iterative approach to generate k-length itemsets and update the frequent itemsets list until there when are no more frequent sets
# being generated from the provide possible itemsets

def get_freq_itemsets(shop_data,min_support):
    freq_itemsets = {}
    k = 1

    while True:
        # generates all possible k-itemsets
        k_itemsets = [subset for subset in get_combinations(shop_data["items"],k)]
        k_itemsets = list(frozenset(k_itemsets))
        print(f"{k}-itemsets generated: {len(k_itemsets)}")

        # checks to see which itemsets are frequent and stops if an empty list is provided
        freq_k_itemsets = get_k_freq_itemsets(shop_data,k_itemsets,min_support)
        if not freq_k_itemsets:  # stopping condition
            break

        freq_itemsets.update(freq_k_itemsets)

        prev_itemsets = frozenset(freq_k_itemsets.keys())
        k += 1

    return freq_itemsets

In [ ]:
# generate association rules from the given frequent itemsets based on the minimum confidence provided

def get_rules(freq_itemsets,shop_data,min_conf):
    assoc_rules = {}

    for itemset,support in freq_itemsets.items():
        if len(itemset) <= 1:
            continue
        subsets = [subset for k in range(1,len(itemset)) for subset in get_combinations(list(itemset),k)]
        for lhs in subsets:
            rhs = itemset - lhs
            lhs_support = freq_itemsets[lhs]
            conf = support/lhs_support
            if conf >= min_conf:
                assoc_rules[(lhs,rhs)] = (support,conf)

    return assoc_rules

In [ ]:
# execute the functions and calculate time

start_time1 = time.time()

freq_itemsets1 = get_freq_itemsets(shop_data,min_support)
assoc_rules1 = get_rules(freq_itemsets1,shop_data,min_conf)

end_time1 = time.time()

1-itemsets generated: 9
2-itemsets generated: 36
3-itemsets generated: 84
4-itemsets generated: 126


In [ ]:
print(f"{135*'-'}")
print(f"{35*' '}FREQUENT ITEMSETS BRUTE-FORCE (MINIMUM SUPPORT: {min_support*100}%)")
print(f"NO.|{105*' '}ITEMS |   SUPPORT\t")
print(f"{135*'-'}")

i=1
for itemset,support in freq_itemsets1.items():
    items = {item for item in itemset}
    print(f"{' '*(3-len(str(i)))}{i}|{(110-len(str(items)))*' '}{items} | {round(support/len(shop_data['trans']),6)}\t")
    i += 1
print(f"{135*'-'}")

---------------------------------------------------------------------------------------------------------------------------------------
                                   FREQUENT ITEMSETS BRUTE-FORCE (MINIMUM SUPPORT: 40.0%)
NO.|                                                                                                         ITEMS |   SUPPORT	
---------------------------------------------------------------------------------------------------------------------------------------
  1|                                                                               {'Head First Java 2nd Edition'} | 0.4	
  2|                                                                                        {"A Beginner's Guide"} | 0.55	
  3|                                                                   {'Android Programming: The Big Nerd Ranch'} | 0.65	
  4|                                                                                          {'Java For Dummies'} | 0.65	
  5|               

In [ ]:
print(f"{35*' '}ASSOCIATION RULES BRUTE-FORCE (MINIMUM CONFIDENCE: {min_conf*100}%)")
print("\n")
i = 1
for (lhs,rhs),(support,conf) in assoc_rules1.items():
    lhs_items = {item for item in lhs}
    rhs_items = {item for item in rhs}
    print(f"{' '*(3-len(str(i)))}{i}: {lhs_items} -> {rhs_items}{(100-len(str(lhs)+str(rhs)))*' '}\
            Support: {round(support/len(shop_data['trans']),6)}, Confidence: {round(conf,6)}")
    i += 1
print(f"{135*'-'}")

                                   ASSOCIATION RULES BRUTE-FORCE (MINIMUM CONFIDENCE: 60.0%)


  1: {"A Beginner's Guide"} -> {'Java For Dummies'}                                                Support: 0.45, Confidence: 0.818182
  2: {'Java For Dummies'} -> {"A Beginner's Guide"}                                                Support: 0.45, Confidence: 0.692308
  3: {'Android Programming: The Big Nerd Ranch'} -> {'Java For Dummies'}                           Support: 0.45, Confidence: 0.692308
  4: {'Java For Dummies'} -> {'Android Programming: The Big Nerd Ranch'}                           Support: 0.45, Confidence: 0.692308
  5: {'Java: The Complete Reference'} -> {"A Beginner's Guide"}                                    Support: 0.45, Confidence: 0.9
  6: {"A Beginner's Guide"} -> {'Java: The Complete Reference'}                                    Support: 0.45, Confidence: 0.818182
  7: {'Java: The Complete Reference'} -> {'Java For Dummies'}                                      S

### Python Package Implementation

In [ ]:
# structure the data according to package requirements and execute the function

df = pd.DataFrame([
    {item: True if item in trans else False for item in shop_data["items"]}
    for trans in shop_data["trans"]
])

start_time2 = time.time()

freq_itemsets2 = apriori(df, min_support=min_support, use_colnames=True)
if not freq_itemsets2.empty:
    assoc_rules2 = association_rules(freq_itemsets2, metric="confidence", min_threshold=min_conf)
    assoc_rules2 = assoc_rules2.loc[:,["antecedents","consequents","support","confidence"]]
else:
    assoc_rules2 = pd.DataFrame([],columns=["antecedents","consequents","support","confidence"])

end_time2 = time.time()

In [ ]:
print(f"{135*'-'}")
print(f"{35*' '}FREQUENT ITEMSETS PY PACKAGE (MINIMUM SUPPORT: {min_support*100}%)")
print(f"NO.|{105*' '}ITEMS |   SUPPORT\t")
print(f"{135*'-'}")

i=1
for support,itemset in freq_itemsets2.values:
    items = {item for item in itemset}
    print(f"{' '*(3-len(str(i)))}{i}|{(110-len(str(items)))*' '}{items} | {support}\t")
    i += 1
print(f"{135*'-'}")

---------------------------------------------------------------------------------------------------------------------------------------
                                   FREQUENT ITEMSETS PY PACKAGE (MINIMUM SUPPORT: 40.0%)
NO.|                                                                                                         ITEMS |   SUPPORT	
---------------------------------------------------------------------------------------------------------------------------------------
  1|                                                                   {'Android Programming: The Big Nerd Ranch'} | 0.65	
  2|                                                                              {'Java: The Complete Reference'} | 0.5	
  3|                                                                               {'Head First Java 2nd Edition'} | 0.4	
  4|                                                                                        {"A Beginner's Guide"} | 0.55	
  5|                 

In [ ]:
print(f"{35*' '}ASSOCIATION RULES PY PACKAGE (MINIMUM CONFIDENCE: {min_conf*100}%)")
print("\n")
i = 1
for lhs,rhs,support,conf in assoc_rules2.values:
    lhs_items = {item for item in lhs}
    rhs_items = {item for item in rhs}
    print(f"{' '*(3-len(str(i)))}{i}: {lhs_items} -> {rhs_items}{(100-len(str(lhs)+str(rhs)))*' '}\
            Support: {round(support,6)}, Confidence: {round(conf,6)}")
    i += 1
print(f"{135*'-'}")

                                   ASSOCIATION RULES PY PACKAGE (MINIMUM CONFIDENCE: 60.0%)


  1: {'Android Programming: The Big Nerd Ranch'} -> {'Java For Dummies'}                           Support: 0.45, Confidence: 0.692308
  2: {'Java For Dummies'} -> {'Android Programming: The Big Nerd Ranch'}                           Support: 0.45, Confidence: 0.692308
  3: {'Java: The Complete Reference'} -> {"A Beginner's Guide"}                                    Support: 0.45, Confidence: 0.9
  4: {"A Beginner's Guide"} -> {'Java: The Complete Reference'}                                    Support: 0.45, Confidence: 0.818182
  5: {'Java: The Complete Reference'} -> {'Java For Dummies'}                                      Support: 0.5, Confidence: 1.0
  6: {'Java For Dummies'} -> {'Java: The Complete Reference'}                                      Support: 0.5, Confidence: 0.769231
  7: {"A Beginner's Guide"} -> {'Java For Dummies'}                                                Support: 

### Time Comparison

In [ ]:
print(f"Time taken to run the brute force algorithm is {round(end_time1-start_time1,6)}s.")
print(f"Time taken to run the python package is {round(end_time2-start_time2,6)}s.")

Time taken to run the brute force algorithm is 0.001495s.
Time taken to run the python package is 0.007344s.
